#**Step 01: Install All the Required Packages**

In [ ]:
!pip install langchain -q
#For API Calls
!pip install huggingface_hub -q
!pip install transformers -q
!pip install accelerate -q
!pip install  bitsandbytes -q

#**Step 02: Import All the Required Libraries**

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from huggingface_hub import InferenceApi
import os

#**Step 03: Setting the Environment**

In [ ]:
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'hf'
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')


#**Step 04: Approach 1:  Access Models Hosted on Hugging Face Through API**

#**Text2Text Generation Models | Seq2Seq Models | Encoder-Decoder Models**

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}"
)

In [ ]:
model = 'google/flan-t5-large'
llm = InferenceApi(repo_id=model, task="text2text-generation", token = HUGGINGFACEHUB_API_TOKEN)

In [ ]:
chain = LLMChain(llm=HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':0, 'max_length':64},huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN),prompt = prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
chain.invoke("colorful socks")

{'product': 'colorful socks', 'text': 'sock mania'}

In [ ]:
prompt = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me about famous footballer {name}"
)

In [ ]:
chain = LLMChain(llm=HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':0.1, 'max_length':64},huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN),prompt = prompt)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [ ]:
chain.run("Messi")

'Messi is a footballer who plays for Argentina.'

In [ ]:
prompt = PromptTemplate(
    input_variables=["cusine"],
    template="Can you tell me food items for a  {cusine} restuarant"
)

In [ ]:
chain = LLMChain(llm=HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':0, 'max_length':64},huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN),
                 prompt = prompt)

In [ ]:
chain.run("Pakistani")

#**Apprach 01: Text Generation Models | Decoder Only Models**

In [ ]:
prompt = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me about famous footballer {name}"
)

In [ ]:
chain = LLMChain(llm=HuggingFaceHub(repo_id='tiiuae/falcon-7b', model_kwargs={'temperature':0.1, 'max_length':64}, huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN),
                 prompt = prompt)

In [ ]:
chain.run("Messi")

'?\nMessi is a famous footballer who plays for FC Barcelona. He is the best player'

#**Step 05: Approach 02: Download Model Locally (Create Pipelines)**

#**Import All the Required Libraries**

In [ ]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


In [ ]:
model_id = 'google/flan-t5-large'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto')

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=128)

In [ ]:
local_llm = HuggingFacePipeline(pipeline=pipeline)

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}"
)

In [ ]:
chain = LLMChain(llm=local_llm,prompt = prompt)

In [ ]:
chain.run("colorful socks")

'sock mania'

In [ ]:
prompt = PromptTemplate(
    input_variables=["name"],
    template="Can you tell me about famous footballer {name}"
)

In [ ]:
chain = LLMChain(llm=local_llm,prompt = prompt)

In [ ]:
chain.run("Messi")

'Messi is a footballer who plays for Argentina.'